<a href="https://colab.research.google.com/github/gauravraidata/IITJ-projects/blob/main/GPU_Ass1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
import cudf
import cupy as cp
from cuml.ensemble import RandomForestClassifier as cuRF
from cuml.model_selection import train_test_split
from cuml.model_selection import GridSearchCV as cuGridSearchCV
from cuml.metrics import accuracy_score
from cuml.preprocessing import StandardScaler
from sklearn.model_selection import KFold

In [15]:
try:
    dbt_df_gpu = cudf.read_csv("/content/drive/MyDrive/Assignment files/Ass2 ML/diabetes (1).csv")
except FileNotFoundError:
    print("Error: diabetes (1).csv not found. Creating dummy GPU data.")
    # Create dummy data directly on the GPU
    dummy_data = cp.random.rand(768, 8)
    dummy_target = cp.random.randint(0, 2, 768)

    dbt_df_gpu = cudf.DataFrame(dummy_data,
                                columns=['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness',
                                         'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age'])
    dbt_df_gpu['Outcome'] = dummy_target

print("Data loaded onto GPU successfully.")

Data loaded onto GPU successfully.


In [16]:
X_gpu = dbt_df_gpu.drop('Outcome', axis=1)
y_gpu = dbt_df_gpu['Outcome']

scaler_gpu = StandardScaler()
scaled_X_gpu = scaler_gpu.fit_transform(X_gpu)

In [17]:
X_train_gpu, X_test_gpu, y_train_gpu, y_test_gpu = train_test_split(
    scaled_X_gpu, y_gpu, test_size=0.2, random_state=42
)

print(f"Training set size: {X_train_gpu.shape[0]}, Test set size: {X_test_gpu.shape[0]}")

Training set size: 615, Test set size: 153


In [23]:
print("\n--- Model: Random Forest (Hyperparameter Tuning on GPU) ---")

param_grid = {
    'n_estimators': [50, 100, 200, 400],
    'max_depth': [10, 20, 30],
    'max_features': [3, 5, 8, 'auto', 0.5],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False],
    'split_criterion': [0, 1]
}

N_SPLITS = 3
# --- Use sklearn's KFold ---
kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=42)

train_indices_cp = cp.arange(X_train_gpu.shape[0])

# --- THIS IS THE FIX ---
# kf.split() is an sklearn function, so it needs a NumPy array.
# We use .get() to pull the tiny index array [0, 1, 2...] to the CPU.
# The resulting train_index/val_index will be NumPy arrays.
cv_splits_gpu = list(kf.split(train_indices_cp.get()))


--- Model: Random Forest (Hyperparameter Tuning on GPU) ---


In [24]:
print("Performing manual Grid Search on GPU with explicit CV splits...")

best_score = 0
best_params = {}

for n_estimators in param_grid['n_estimators']:
    for max_depth in param_grid['max_depth']:
        for max_features in param_grid['max_features']:
            for min_samples_split in param_grid['min_samples_split']:
                for min_samples_leaf in param_grid['min_samples_leaf']:
                    for bootstrap in param_grid['bootstrap']:

                        fold_scores = []
                        # Iterate through each cross-validation split
                        for train_index, val_index in cv_splits_gpu:
                            # Get training and validation data for the current fold
                            # .iloc is correct for cudf.DataFrame and cudf.Series
                            X_train_fold, X_val_fold = X_train_gpu.iloc[train_index], X_train_gpu.iloc[val_index]
                            y_train_fold, y_val_fold = y_train_gpu.iloc[train_index], y_train_gpu.iloc[val_index]

                            # Initialize and train cuML RF with *all* parameters
                            model_rf_fold = cuRF(
                                n_estimators=n_estimators,
                                max_depth=max_depth,
                                max_features=max_features,
                                min_samples_split=min_samples_split,
                                min_samples_leaf=min_samples_leaf,
                                bootstrap=bootstrap,
                                random_state=42
                            )
                            model_rf_fold.fit(X_train_fold, y_train_fold)

                            # Predict and evaluate on the validation fold
                            y_pred_fold = model_rf_fold.predict(X_val_fold)

                            # Need to convert cudf.Series to cupy array for accuracy_score
                            # Or ensure both are cupy arrays.
                            # If y_val_fold is cudf, score = accuracy_score(y_val_fold, y_pred_fold) is fine.
                            score = accuracy_score(y_val_fold, y_pred_fold)

                            # Convert score (which might be a cupy float) to a standard Python float
                            fold_scores.append(float(score))

                        # Calculate the average score across folds
                        mean_score = sum(fold_scores) / len(fold_scores)

                        # Check if the current parameters are better
                        if mean_score > best_score:
                            best_score = mean_score
                            best_params = {
                                'n_estimators': n_estimators,
                                'max_depth': max_depth,
                                'max_features': max_features,
                                'min_samples_split': min_samples_split,
                                'min_samples_leaf': min_samples_leaf,
                                'bootstrap': bootstrap
                            }


Performing manual Grid Search on GPU with explicit CV splits...


/usr/local/lib/python3.12/dist-packages/cuml/internals/api_decorators.py:216: FutureWarning: `max_features='auto'` has been deprecated in 24.06 and will be removed in 25.08. To keep the past behaviour and silence this warning, explicitly set `max_features='sqrt'`.
  ret = func(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/cuml/internals/api_decorators.py:216: FutureWarning: `max_features='auto'` has been deprecated in 24.06 and will be removed in 25.08. To keep the past behaviour and silence this warning, explicitly set `max_features='sqrt'`.
  ret = func(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/cuml/internals/api_decorators.py:216: FutureWarning: `max_features='auto'` has been deprecated in 24.06 and will be removed in 25.08. To keep the past behaviour and silence this warning, explicitly set `max_features='sqrt'`.
  ret = func(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/cuml/internals/api_decorators.py:216: FutureWarning: `max_features='auto'` 

In [25]:
print("\nBest parameters found by manual Grid Search:")
print(best_params)
print("\nBest cross-validation accuracy score (mean across folds):")
print(best_score)


Best parameters found by manual Grid Search:
{'n_estimators': 50, 'max_depth': 10, 'max_features': 'auto', 'min_samples_split': 10, 'min_samples_leaf': 4, 'bootstrap': False}

Best cross-validation accuracy score (mean across folds):
0.7723577235772359


In [26]:
best_model_gpu = cuRF(random_state=42, **best_params)
best_model_gpu.fit(X_train_gpu, y_train_gpu)

/usr/local/lib/python3.12/dist-packages/cuml/internals/api_decorators.py:216: FutureWarning: `max_features='auto'` has been deprecated in 24.06 and will be removed in 25.08. To keep the past behaviour and silence this warning, explicitly set `max_features='sqrt'`.
  ret = func(*args, **kwargs)


RandomForestClassifier()

In [27]:
y_pred_gpu = best_model_gpu.predict(X_test_gpu)

# Calculate final accuracy on the test set
acc_rf_gpu = accuracy_score(y_test_gpu, y_pred_gpu)

print(f"Accuracy on Test Set: {acc_rf_gpu:.4f}")
print(f"Error on Test Set: {1 - acc_rf_gpu:.4f}\n")

Accuracy on Test Set: 0.7778
Error on Test Set: 0.2222

